In [0]:
!pip install kafka-python
!pip install murmurhash

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
#Importamos lo que ocuparemos
#Guardamos la API key para recibir las noticias del sitio cryptocompare

from kafka import KafkaProducer
import requests
import time
import datetime
import codecs

api_key = 'd65d7dda1cfde9f7a7f6f517d624d8a0c575f186647af620f9170e5bd5ef7092'
api_url = "https://min-api.cryptocompare.com/data/v2/news/?lang=EN"
producer = KafkaProducer(bootstrap_servers='localhost:9092')

# Noticias de Criptomonedas

Existen diferentes sitios de noticias que habla sobre criptomonedas, nuestro proximo analisis será extraer las categorías de estas noticias para recomendar a usuarios inscritos noticias de diferentes temas a traves del filtro de Bloom.

Para las palabras más usadas, primero eliminaremos las stopwords:

In [0]:
#Descagamos las stopwords y las guardamos en una lista

!wget -qO- -O stopwords_english.txt \
         https://raw.githubusercontent.com/pan-webis-de/authorid/master/data/stopwords_english.txt

stopwords = {}
for line in codecs.open('stopwords_english.txt', encoding = "utf-8"):
    stopwords[line.rstrip()] = True

Una vez eliminado las stopwords, normalizaremos nuestro texto para posteriormente pasarlo a la cola de mensajes de kafka:

In [0]:
#Utilizamos un while sin condición para obtener las noticias

while True:
    response = requests.get(api_url)
    json_ = response.json()    
    
    #Utilizamos un ciclo for para obtener los títulos de las noticias
    for i in range(len(json_['Data'])):
        title = json_['Data'][i]['title']
        # Transformando a minusculas el título
        title = title.lower()
        # Removiendo las stopwords del título             
        title_no_stopword = [word for word in title.split(' ') if not stopwords.get(word, False)]
        new_title = ' '.join(title_no_stopword)
        print(new_title.encode('utf-8'))                          
        producer.send("noticias", new_title.encode('utf-8'))  
        time.sleep(3)

b'shopify dives deeper crypto payments crypto.com deal; shares jump'
b'avalanche (avax) team explains subnets scalability solutions'
b'luna crash: blockchain firm losses $3.5 billion terra (luna) crumbles'
b'micha\xc3\xabl van de poppe\xe2\x80\x99s bitcoin today, btc surge soon?'
b'tether report shows 17% reduction commercial paper'
b'breaking: ripple (xrp) throws $100 million cut carbon emission'
b"indian government's \xe2\x80\x98blockchain crypto\xe2\x80\x99 stance highlights lack understanding"
b'bitcoin entering accumulation range \xe2\x80\x93 arrows capital co-founder'
b'avalanche slips 12% spite skyrocketing active wallet count'
b'bitcoin price hit low $7,000 time high: economist'
b'bitcoin dominance strong 6 months'
b'bitcoin, binance coin, xrp, solana daily price analyses \xe2\x80\x93 19 morning price prediction'
b'bitcoin tests usd 30k, ethereum usd 2k, altcoins move higher'
b'shouldn\xe2\x80\x99t panic sell crypto whales liquidate'
b'bitcoin education launching 40 high school

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e588c84d-c012-4c07-8593-b08d1bf1b087' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>